In [4]:
import gradio as gr 
import requests 
from openai import OpenAI 
from bs4 import BeautifulSoup 
from dotenv import load_dotenv
import os

IS_LOCAL=True
MODEL=""

if(IS_LOCAL):
    openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
    MODEL="llama3.2"
    !ollama pull $MODEL
else:
    MODEL="gpt-4o-mini"
    load_dotenv(override=True)
    openai_key = os.getenv("OPENAI_API_KEY")
    openai = OpenAI()
    if openai_key:
        print("OPENAI_API_KEY loaded with success! ")
    else: 
        print("OPENAI_API_KEY not set!")



pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
class Website:
    url: str 
    site: str 
    text: str
    
    def __init__(self, url):
        self.url = url 
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "Not title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose() 
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


    
            
    

In [6]:
system_message = """You are an assistant that analyzes the contents of a company website landing page \
 and create a short brochure about the company for prospective customers, investors and recruiters. Respond in markdown.

"""

In [7]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message },
        {"role": "user", "content": prompt }
       
    ]
    stream = openai.chat.completions.create(
          model=MODEL,
          messages=messages,
          stream=True
        )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result
    

In [8]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"""Please generate a company brochure for {company_name}. here is their landing page:\n
    """
    prompt += Website(url).get_contents()
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "CLAUDE":
        result = ""
        print("Claude was not implemented")
    else:
        raise ValueError("Unknown model")
    yield from result
    

In [10]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name"),
        gr.Textbox(label="Landing Page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select Model")
    ], 
    outputs=[gr.Markdown(label="Brochure:")]
)

view.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
